# Langchain #1: Prompt Template

출처: https://teddylee777.github.io/langchain/langchain-tutorial-01/

# Langchain의 주요 기능
* 사용자의 문맥을 정확히 인식
* 문맥에 기반하여 어떤 대답을 할지, 어떤 액션을 취할지 추론 가능
* 각 작업에 알맞은 구성 요소가 준비되어 있음 -> 손쉬운 사용 가능

In [ ]:
!pip install openai langchain

In [2]:
import os
os.environ['OPENAI_API_KEY'] = '안알려줌'

In [11]:
import openai
for m in openai.models.list().data:
    print(m.id)

gpt-3.5-turbo-0301
gpt-4-1106-preview
dall-e-3
dall-e-2
whisper-1
gpt-3.5-turbo-0125
text-embedding-ada-002
tts-1-hd-1106
tts-1-hd
davinci-002
babbage-002
text-embedding-3-small
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-16k
gpt-3.5-turbo
gpt-4
gpt-4-0613
gpt-4-vision-preview
gpt-3.5-turbo-0613
text-embedding-3-large
gpt-3.5-turbo-1106
gpt-4-0125-preview
gpt-4-turbo-preview
tts-1-1106
gpt-3.5-turbo-instruct
tts-1
gpt-3.5-turbo-instruct-0914


# ChatOpenAI

*  `temperature`: 0과 2 사이 값, 높을수록 무작위한 출력, 낮을수록 결정론적 추론.
* `max_tokens`: 생성할 토큰의 최대 개수.
* `model_name`: 적용가능한 모델 리스트.

In [19]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0.8, max_tokens=2048, model_name='gpt-3.5-turbo')
question = "책 '꿈의 해석'의 저자는 누구야?"
print(f"[답변]: {llm.predict(question)}")

[답변]: 책 '꿈의 해석'의 저자는 지그문트 프로이드(Sigmund Freud) 이다.


# 프롬프트 템플릿의 활용

* `PromptTemplate`: `template`은 템플릿 문자열로, 이 중 변수를 중괄호로 표시한다. `input_variables`는 중괄호에 들어갈 변수 이름의 리스트다.
* `LLMChain`: `PromptTemplate`과 연결된 체인객체를 생성한다.

In [20]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
template = '{book}의 저자는 누구야?'
prompt = PromptTemplate(template=template, input_variables=['book'])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [21]:
# run: PromptTemplate 실행
print(llm_chain.run(book="해리 포터와 혼혈 왕자"))
print(llm_chain.run(book="퍼시 잭슨과 번개 도둑"))

해리 포터와 혼혈 왕자의 저자는 J.K. 롤링(J.K. Rowling)이다.
퍼시 잭슨과 번개 도둑의 저자는 리키 리오던이다.


In [22]:
# apply: 여러 입력을 한 번에 실행
input_list = [
    {"book": "초콜릿 전쟁"},
    {"book": "정의란 무엇인가"},
    {"book": "아메리칸 프로메테우스"}
]
llm_chain.apply(input_list)

[{'text': '초콜릿 전쟁의 저자는 로버트 코먼이다.'},
 {'text': '"정의란 무엇인가"의 저자는 플라톤입니다.'},
 {'text': '아메리칸 프로메테우스의 저자는 미국의 작가 Ayn Rand(앤 랜드)입니다.'}]

In [23]:
result = llm_chain.apply(input_list)
for r in result:
    print(r['text'])

초콜릿 전쟁의 저자는 로알드 달입니다.
정의란 무엇인가의 저자는 누구인지 명확한 정보가 없습니다. "정의란 무엇인가"라는 주제에 대한 저자는 여러 사람들이 쓴 책, 논문, 에세이 등이 있을 수 있습니다. 일반적으로 이 주제에 대한 다양한 저자와 관점이 존재합니다. 따라서 질문에 대한 단정적인 답변을 제공하기는 어렵습니다.
아메리칸 프로메테우스의 저자는 Andrew Ervin이다.


In [24]:
# generate: apply와 다르게, 문자열 대신 LLMResult을 return.
gen_result = llm_chain.generate(input_list)
print(gen_result)

generations=[[ChatGeneration(text='초콜릿 전쟁의 저자는 로아루 달입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='초콜릿 전쟁의 저자는 로아루 달입니다.'))], [ChatGeneration(text='"정의란 무엇인가"의 저자는 플라톤(Plato)이다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='"정의란 무엇인가"의 저자는 플라톤(Plato)이다.'))], [ChatGeneration(text='아메리칸 프로메테우스의 저자는 애이바 실버스틴이다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='아메리칸 프로메테우스의 저자는 애이바 실버스틴이다.'))]] llm_output={'token_usage': {'completion_tokens': 87, 'prompt_tokens': 87, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo'} run=[RunInfo(run_id=UUID('327ffdca-0172-4803-ab2b-250df99f9591')), RunInfo(run_id=UUID('4863e624-2611-41ab-9af0-5c5aac0f2e5c')), RunInfo(run_id=UUID('9c624b0f-286f-4bcb-9126-51fa32fef91e'))]


In [25]:
gen_result.generations

[[ChatGeneration(text='초콜릿 전쟁의 저자는 로아루 달입니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='초콜릿 전쟁의 저자는 로아루 달입니다.'))],
 [ChatGeneration(text='"정의란 무엇인가"의 저자는 플라톤(Plato)이다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='"정의란 무엇인가"의 저자는 플라톤(Plato)이다.'))],
 [ChatGeneration(text='아메리칸 프로메테우스의 저자는 애이바 실버스틴이다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='아메리칸 프로메테우스의 저자는 애이바 실버스틴이다.'))]]

In [26]:
# 토큰 사용량
gen_result.llm_output

{'token_usage': {'completion_tokens': 87,
  'prompt_tokens': 87,
  'total_tokens': 174},
 'model_name': 'gpt-3.5-turbo'}

In [27]:
# run Id
gen_result.run

[RunInfo(run_id=UUID('327ffdca-0172-4803-ab2b-250df99f9591')),
 RunInfo(run_id=UUID('4863e624-2611-41ab-9af0-5c5aac0f2e5c')),
 RunInfo(run_id=UUID('9c624b0f-286f-4bcb-9126-51fa32fef91e'))]

# 여러 변수를 정의

In [28]:
template = '{object1}와 {object2} 중 뭐가 더 무거워?'
prompt = PromptTemplate(template=template, input_variables=['object1', 'object2'])
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(object1="코끼리", object2="돼지"))

코끼리가 돼지보다 훨씬 더 무겁습니다. 코끼리는 일반적으로 5,000킬로그램 이상의 몸무게를 가지며, 최대 7,000킬로그램까지 살 수 있습니다. 반면에 돼지는 대개 50-350킬로그램 정도의 몸무게를 가집니다.


In [30]:
input_list = [
    {"object1": "동전", "object2": "지폐"},
    {"object1": "닌텐도 스위치", "object2": "PS5"},
    {"object1": "갤럭시 S20", "object2": "아이폰 11 Pro"}
]
result = llm_chain.apply(input_list)
for r in result:
    print(r['text'].strip())

동전과 지폐는 모두 다양한 무게를 가지고 있습니다. 그러나 일반적으로 동전이 지폐보다 더 무겁습니다. 동전은 금속으로 만들어져 있어서 더 높은 밀도와 무게를 가지고 있습니다. 그러나 이는 모든 동전과 모든 지폐에 적용되는 것은 아니며, 각 나라에서 사용하는 동전과 지폐의 무게는 서로 다를 수 있습니다.
PS5이 더 무겁습니다. PS5의 무게는 약 4.5kg입니다. 반면, 닌텐도 스위치의 무게는 약 0.4kg입니다.
갤럭시 S20은 약 163g이며, 아이폰 11 Pro는 약 188g입니다. 따라서 아이폰 11 Pro가 더 무겁습니다.


In [31]:
# streaming: 실시간 답변
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = ChatOpenAI(temperature=0, max_tokens=2048,
                 model_name='gpt-3.5-turbo', streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
                 )
question = "친구랑 싸웠을 때 화해할 수 있는 방법을 있는 대로 말해줄래?"
reponse = llm.predict(question)

물론입니다! 친구와 화해하는 방법은 다양하지만, 몇 가지 제안을 해드릴게요.

1. 이해와 소통: 상황을 이해하고 상대방의 감정을 들어주는 것이 중요합니다. 서로의 의견을 솔직하게 표현하고, 서로를 듣고 이해하는 노력을 기울여보세요.

2. 사과와 용서: 만약 자신이 잘못했다면 사과하는 것이 중요합니다. 진심 어린 사과는 상대방에게 용서와 화해의 기회를 줄 수 있습니다. 또한, 상대방이 사과를 한다면 용서하는 마음을 갖는 것도 중요합니다.

3. 중재자의 도움: 상황이 심각하거나 양측이 갈등을 해결하기 어려울 때는 중재자의 도움을 받을 수 있습니다. 중립적인 친구나 가족의 도움을 받아 갈등을 조정하고 화해할 수 있습니다.

4. 시간과 공간: 때로는 갈등 상황에서 잠시 시간과 공간을 가지는 것이 도움이 될 수 있습니다. 감정이 가라앉고 상황을 재평가할 수 있는 시간을 가지고, 상대방과의 접촉을 피해보세요.

5. 새로운 시작: 화해 후에는 새로운 시작을 할 수 있도록 노력해보세요. 과거의 실수나 갈등을 되풀이하지 않고, 새로운 관계를 구축하고 서로를 존중하는 태도를 가지는 것이 중요합니다.

이러한 방법들을 참고하여 친구와의 갈등을 해결하고 화해할 수 있기를 바랍니다. 하지만 모든 상황에 적용되는 법칙은 아니므로, 상황에 맞게 적절한 방법을 선택하는 것이 중요합니다.

# HuggingFace Model과 연동

In [33]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = '안알려줌'

리더보드를 통해 자신에게 필요한 모형을 찾는 것이 중요하다.

* https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
* https://huggingface.co/spaces/upstage/open-ko-llm-leaderboard


In [37]:
from langchain.llms import HuggingFaceHub
repo_id = 'skt/kogpt2-base-v2'
question = "어젯밤에 우리 아빠가"
template = """{question}"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm = HuggingFaceHub(
    repo_id=repo_id,
    model_kwargs={"temperature": 1, "max_length": 128}
)
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(question=question))

어젯밤에 우리 아빠가 그게 아니라 저기 그니까 아빠? 그래갖고 이게 어딘가 좀 궁금해서 네. 또 아빠도 궁금해서 어~ 그래갖고 어어 그래갖고 뭐 저기 좀
